# DoD Contracts - Top Contract Winning Vendors

## Aims
The aim of this notebook is to identify the top vendors in terms of contract wins and contract value. In this specific context, we will be looking at the US Department of Defense.

In [4]:
import pandas as pd
import requests

def get_awards(year, page, limit=100):

    start_date = f"{year}-01-01"
    end_date = f"{year + 1}-01-01"

    # Define the API endpoint URL
    base_url = "https://api.usaspending.gov/api/v2/search/spending_by_award/"

    url = f"{base_url}"

    agency_object = {
        "type": "awarding",  
        "tier": "toptier",   
        "name": "Department of Defense"  
    }
    
    # Construct the payload
    payload =   {
      "subawards": False,
      "limit": limit,
      "page": page,
      "order": "desc",
      "sort": "Award Amount",
      "filters": {
          "award_type_codes": ["A", "B", "C", "D"],
          "time_period": [{"start_date": start_date, "end_date": end_date, "date_type": "new_awards_only"}],
          "agencies": [agency_object]
      },
      "fields": [
          "Award ID",
          "Recipient Name",
          "Start Date",
          "End Date",
          "Award Amount",
          "Awarding Agency",
          "Awarding Sub Agency",
          "Contract Award Type",
          "Award Type",
          "Funding Agency",
          "Funding Sub Agency"
      ]
    }

    # Make the API request
    response = requests.post(url, json=payload)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        return None
    


In [58]:
def get_award_count(year):

    start_date = f"{year}-01-01"
    end_date = f"{year + 1}-01-01"  
    # Define the API endpoint URL
    base_url = "https://api.usaspending.gov/api/v2/search/spending_by_award_count/"

    url = f"{base_url}"

    agency_object = {
        "type": "awarding",  
        "tier": "toptier",   
        "name": "Department of Defense"  
    }

    time_period_object = {
        "date_type": "new_awards_only",
        "start_date": start_date,
        "end_date": end_date
    }
    
    # Construct the payload
    payload =   {
      "filters": {
          "award_type_codes": ["A", "B", "C"],
          "time_period": [time_period_object],
          "agencies": [agency_object]
      }
    }

    # Make the API request
    response = requests.post(url, json=payload)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        return None
    
    


In [20]:
def get_recipients(year):

    start_date = f"{year}-01-01"
    end_date = f"{year + 1}-01-01"  
    # Define the API endpoint URL
    base_url = f"https://api.usaspending.gov/api/v2/award_spending/recipient/?awarding_agency_id=97&fiscal_year={year}"

    url = f"{base_url}"
    print(url)
    
    # Make the API request
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        print(response.content)
        return None
    

In [21]:
data = get_recipients(2021)
data

https://api.usaspending.gov/api/v2/award_spending/recipient/?awarding_agency_id=97&fiscal_year=2021
Failed to retrieve data. Status code: 400
b'{"detail":"Awarding Agency ID provided must correspond to a toptier agency"}'


In [59]:
def get_award_counts(start_year, end_year):
    award_count = {}
    for year in range(start_year, end_year + 1):
        count_data = get_award_count(year)
        award_count[year] = count_data['results']['contracts']
    return award_count


In [60]:
award_counts = get_award_counts(2021, 2023)
award_counts

{2021: 3757308, 2022: 3746169, 2023: 2234664}

In [64]:
df = pd.DataFrame.from_dict(data['results'])

In [66]:
df

,internal_id,Award ID,Recipient Name,Start Date,End Date,Award Amount,Awarding Agency,Awarding Sub Agency,Contract Award Type,Award Type,Funding Agency,Funding Sub Agency,awarding_agency_id,agency_slug,generated_internal_id
0,146200082,W58P0522C0001,PFIZER INC,2021-11-17,2023-03-31,1.255000e+10,Department of Defense,Department of the Army,DEFINITIVE CONTRACT,None,Department of Defense,Department of the Army,1173,department-of-defense,CONT_AWD_W58P0522C0001_9700_-NONE-_-NONE-
1,130218147,W15QKN21C0014,REGENERON PHARMACEUTICALS INC,2021-01-12,2022-07-31,5.565000e+09,Department of Defense,Department of the Army,DEFINITIVE CONTRACT,None,Department of Defense,Department of the Army,1173,department-of-defense,CONT_AWD_W15QKN21C0014_9700_-NONE-_-NONE-
2,139947836,W58RGZ21C0015,THE BOEING COMPANY,2021-06-18,2027-12-31,2.518933e+09,Department of Defense,Department of the Army,DEFINITIVE CONTRACT,None,Department of Defense,Department of the Army,1173,department-of-defense,CONT_AWD_W58RGZ21C0015_9700_-NONE-_-NONE-
3,133631054,N0002421C2106,HUNTINGTON INGALLS INCORPORATED,2021-02-19,2025-08-03,2.362717e+09,Department of Defense,Department of the Navy,DEFINITIVE CONTRACT,None,Department of Defense,Department of the Navy,1173,department-of-defense,CONT_AWD_N0002421C2106_9700_-NONE-_-NONE-
4,139707069,W911QY21C0031,MERCK SHARP & DOHME LLC,2021-06-07,2023-07-13,2.205001e+09,Department of Defense,Department of the Army,DEFINITIVE CONTRACT,None,Department of Defense,Department of the Army,1173,department-of-defense,CONT_AWD_W911QY21C0031_9700_-NONE-_-NONE-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,134182679,N6247821F4079,HENSEL PHELPS CONSTRUCTION CO,2021-03-04,2025-03-31,2.618725e+08,Department of Defense,Department of the Navy,DELIVERY ORDER,None,Department of Defense,Department of the Navy,1173,department-of-defense,CONT_AWD_N6247821F4079_9700_N6247820D4003_9700
96,138613791,FA861121F0024,LOCKHEED MARTIN CORPORATION,2021-04-30,2025-04-25,2.608876e+08,Department of Defense,Department of the Air Force,DELIVERY ORDER,None,Department of Defense,Department of the Air Force,1173,department-of-defense,CONT_AWD_FA861121F0024_9700_FA861113D2850_9700
97,140372326,N6247321F4921,"RQ CONSTRUCTION, LLC",2021-06-24,2025-10-14,2.573348e+08,Department of Defense,Department of the Navy,DELIVERY ORDER,None,Department of Defense,Department of the Navy,1173,department-of-defense,CONT_AWD_N6247321F4921_9700_N6247321D1211_9700
98,147228242,N0002422C4490,METRO MACHINE CORP,2021-12-22,2023-09-29,2.562135e+08,Department of Defense,Department of the Navy,DEFINITIVE CONTRACT,None,Department of Defense,Department of the Navy,1173,department-of-defense,CONT_AWD_N0002422C4490_9700_-NONE-_-NONE-


In [43]:
award_count

{'results': {'contracts': 1175500,
  'direct_payments': 0,
  'grants': 0,
  'idvs': 0,
  'loans': 0,
  'other': 0},
 'messages': ['For searches, time period start and end dates are currently limited to an earliest date of 2007-10-01.  For data going back to 2000-10-01, use either the Custom Award Download feature on the website or one of our download or bulk_download API endpoints as listed on https://api.usaspending.gov/docs/endpoints.']}